In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier

In [2]:
# read training data

train_data = pd.read_csv('train.csv', delimiter=';')
train_data.head()

,claim_id,fraud,claim_amount,claim_date_registered,claim_date_occured,claim_time_occured,claim_postal_code,claim_cause,claim_liable,claim_num_injured,...,repair_country,repair_sla,policy_date_start,policy_date_next_expiry,policy_date_last_renewed,policy_num_changes,policy_num_claims,policy_premium_100,policy_coverage_1000,policy_coverage_type
0,10000,N,"4895,00",20170101,20161229,NaN,2018,traffic accident,Y,1,...,NaN,N,201604.0,201704.0,201704.0,1,0,4.0,NaN,#000111000
1,10001,N,"3249,81",20170101,20161223,NaN,8820,traffic accident,N,0,...,B,Y,NaN,NaN,NaN,0,0,NaN,NaN,#000000000
2,10002,N,"3242,89",20170101,20161228,NaN,1831,traffic accident,Y,0,...,NaN,N,200704.0,201704.0,201704.0,0,0,20.0,58.0,#111110000
3,10003,N,"2687,00",20170101,20161228,NaN,2610,traffic accident,Y,0,...,NaN,N,199808.0,201708.0,201708.0,0,8,4.0,NaN,#000110000
4,10004,N,"2084,45",20170101,20161229,NaN,1210,traffic accident,Y,0,...,NaN,N,200608.0,201708.0,201708.0,0,9,19.0,15.0,#111110110


In [3]:
# read test data

test_data = pd.read_csv('test.csv', delimiter=';')
test_data.shape

(29955, 76)

In [4]:
# claim variables

train_data['claim_amount'] = train_data['claim_amount'].str.replace(',','.').astype('float64')

train_data['claim_date_registered'] = pd.to_datetime(train_data['claim_date_registered'], format='%Y%m%d')

train_data['claim_date_occured'] = pd.to_datetime(train_data['claim_date_occured'], format='%Y%m%d')

mask_night = (train_data['claim_time_occured'] >= 2200) | (train_data['claim_time_occured'] <= 700)
train_data.loc[~mask_night, 'claim_time_occured'] = 0
train_data.loc[mask_night, 'claim_time_occured'] = 1

postal_code_counts = train_data['claim_postal_code'].value_counts()
train_data = train_data.merge(postal_code_counts, how='left', left_on='claim_postal_code', right_index=True)

train_data['claim_alcohol'].fillna("MISSING", inplace=True)

train_data['claim_language'].fillna("MISSING", inplace=True)
mask = train_data['claim_language'] == 1.0
train_data.loc[mask, 'claim_language'] = "LANG A"
mask = train_data['claim_language'] == 2.0
train_data.loc[mask, 'claim_language'] = "LANG B"

train_data['claim_vehicle_id'].fillna("MISSING", inplace=True)
claim_vehicle_id_count = train_data['claim_vehicle_id'].value_counts()
claim_vehicle_id_count["MISSING"] = 0
train_data = train_data.merge(claim_vehicle_id_count, how='left', 
                              left_on='claim_vehicle_id', right_index=True)

train_data['claim_vehicle_brand'].fillna('MISSING', inplace=True)
claim_vehicle_brand_counts = train_data['claim_vehicle_brand'].value_counts()
claim_vehicle_brand_counts['MISSING'] = 0
train_data = train_data.merge(claim_vehicle_brand_counts, how='left', 
                              left_on='claim_vehicle_brand', right_index=True)


train_data['claim_vehicle_type'].fillna('MISSING', inplace=True)

train_data['claim_vehicle_date_inuse'].fillna(190001, inplace=True)
train_data['claim_vehicle_date_inuse'] = train_data['claim_vehicle_date_inuse'].astype(int)
mask = (train_data['claim_vehicle_date_inuse'] > 220000)
train_data.loc[mask, 'claim_vehicle_date_inuse'] = 190001
train_data['claim_vehicle_date_inuse'] = pd.to_datetime(train_data['claim_vehicle_date_inuse'].astype(str), 
                                                        format='%Y%m')

train_data['claim_vehicle_cyl'].fillna(10000, inplace=True)

train_data['claim_vehicle_load'].fillna(500, inplace=True)

train_data['claim_vehicle_fuel_type'].fillna('MISSING', inplace=True)
mask = train_data['claim_vehicle_fuel_type'] == 1.0
train_data.loc[mask, 'claim_vehicle_fuel_type'] = "FUEL A"
mask = train_data['claim_vehicle_fuel_type'] == 2.0
train_data.loc[mask, 'claim_vehicle_fuel_type'] = "FUEL B"

train_data['claim_vehicle_power'].fillna(1000, inplace=True)

In [5]:
# policy variables

policy_holder_id_count = train_data['policy_holder_id'].value_counts()
train_data = train_data.merge(policy_holder_id_count, how='left', 
                              left_on='policy_holder_id', right_index=True)

train_data['policy_holder_postal_code'].fillna(0, inplace=True)
policy_holder_postal_code_counts = train_data['policy_holder_postal_code'].value_counts()
policy_holder_postal_code_counts.loc[0] = 0
train_data = train_data.merge(policy_holder_postal_code_counts, how='left', 
                              left_on='policy_holder_postal_code', right_index=True)

train_data['policy_holder_year_birth'].fillna(1800, inplace=True)

train_data['policy_holder_expert_id'].fillna("MISSING", inplace=True)
policy_holder_expert_id_count = train_data['policy_holder_expert_id'].value_counts()
policy_holder_expert_id_count['MISSING'] = 0
train_data = train_data.merge(policy_holder_expert_id_count, how='left', 
                              left_on='policy_holder_expert_id', right_index=True)

In [6]:
# driver variables

driver_id_count = train_data['driver_id'].value_counts()
train_data = train_data.merge(driver_id_count, how='left', 
                              left_on='driver_id', right_index=True)

train_data['driver_postal_code'].fillna(0, inplace=True)
driver_postal_code_count = train_data['driver_postal_code'].value_counts()
driver_postal_code_count.loc[0] = 0
train_data = train_data.merge(driver_postal_code_count, how='left', 
                              left_on='driver_postal_code', right_index=True)

train_data['driver_year_birth'].fillna(1801, inplace=True)

train_data['driver_expert_id'].fillna("MISSING", inplace=True)
driver_expert_id_count = train_data['driver_expert_id'].value_counts()
driver_expert_id_count['MISSING'] = 0
train_data = train_data.merge(driver_expert_id_count, how='left', 
                              left_on='driver_expert_id', right_index=True)

train_data['driver_vehicle_id'].fillna("MISSING", inplace=True)
driver_vehicle_id_count = train_data['driver_vehicle_id'].value_counts()
driver_vehicle_id_count["MISSING"] = 0
train_data = train_data.merge(driver_vehicle_id_count, how='left', 
                              left_on='driver_vehicle_id', right_index=True)

In [7]:
# third party 1st variables

train_data['third_party_1_id'].fillna("MISSING", inplace=True)
third_party_1_id_count = train_data['third_party_1_id'].value_counts()
third_party_1_id_count["MISSING"] = 0
train_data = train_data.merge(third_party_1_id_count, how='left', 
                              left_on='third_party_1_id', right_index=True)

train_data['third_party_1_postal_code'].fillna(0, inplace=True)
third_party_1_postal_code_count = train_data['third_party_1_postal_code'].value_counts()
third_party_1_postal_code_count[0] = 0
train_data = train_data.merge(third_party_1_postal_code_count, how='left', 
                              left_on='third_party_1_postal_code', right_index=True)

train_data['third_party_1_injured'].fillna("MISSING", inplace=True)

train_data['third_party_1_vehicle_type'].fillna("MISSING", inplace=True)

train_data['third_party_1_form'].fillna("MISSING", inplace=True)

train_data['third_party_1_year_birth'].fillna(1802, inplace=True)

train_data['third_party_1_country'].fillna("MISSING", inplace=True)

train_data['third_party_1_vehicle_id'].fillna("MISSING", inplace=True)
third_party_1_vehicle_id_count = train_data['third_party_1_vehicle_id'].value_counts()
third_party_1_vehicle_id_count["MISSING"] = 0
train_data = train_data.merge(third_party_1_vehicle_id_count, how='left', 
                              left_on='third_party_1_vehicle_id', right_index=True)

train_data['third_party_1_expert_id'].fillna("MISSING", inplace=True)
third_party_1_expert_id_count = train_data['third_party_1_expert_id'].value_counts()
third_party_1_expert_id_count["MISSING"] = 0
train_data = train_data.merge(third_party_1_expert_id_count, how='left', 
                 left_on='third_party_1_expert_id', right_index=True)

In [8]:
# third party rest of variables

mask = train_data['third_party_2_id'].isna()
train_data.loc[mask, 'third_party_2_id'] = 0
train_data.loc[~mask, 'third_party_2_id'] = 1

mask = train_data['third_party_2_postal_code'].isna()
train_data.loc[mask, 'third_party_2_postal_code'] = 0
train_data.loc[~mask, 'third_party_2_postal_code'] = 1

mask = train_data['third_party_2_injured'].isna()
train_data.loc[mask, 'third_party_2_injured'] = 0
train_data.loc[~mask, 'third_party_2_injured'] = 1

mask = train_data['third_party_2_vehicle_type'].isna()
train_data.loc[mask, 'third_party_2_vehicle_type'] = 0
train_data.loc[~mask, 'third_party_2_vehicle_type'] = 1

mask = train_data['third_party_2_form'].isna()
train_data.loc[mask, 'third_party_2_form'] = 0
train_data.loc[~mask, 'third_party_2_form'] = 1

mask = train_data['third_party_2_year_birth'].isna()
train_data.loc[mask, 'third_party_2_year_birth'] = 0
train_data.loc[~mask, 'third_party_2_year_birth'] = 1

mask = train_data['third_party_2_country'].isna()
train_data.loc[mask, 'third_party_2_country'] = 0
train_data.loc[~mask, 'third_party_2_country'] = 1

mask = train_data['third_party_2_vehicle_id'].isna()
train_data.loc[mask, 'third_party_2_vehicle_id'] = 0
train_data.loc[~mask, 'third_party_2_vehicle_id'] = 1

mask = train_data['third_party_2_expert_id'].isna()
train_data.loc[mask, 'third_party_2_expert_id'] = 0
train_data.loc[~mask, 'third_party_2_expert_id'] = 1

mask = train_data['third_party_3_id'].isna()
train_data.loc[mask, 'third_party_3_id'] = 0
train_data.loc[~mask, 'third_party_3_id'] = 1

mask = train_data['third_party_3_postal_code'].isna()
train_data.loc[mask, 'third_party_3_postal_code'] = 0
train_data.loc[~mask, 'third_party_3_postal_code'] = 1

mask = train_data['third_party_3_injured'].isna()
train_data.loc[mask, 'third_party_3_injured'] = 0
train_data.loc[~mask, 'third_party_3_injured'] = 1

mask = train_data['third_party_3_vehicle_type'].isna()
train_data.loc[mask, 'third_party_3_vehicle_type'] = 0
train_data.loc[~mask, 'third_party_3_vehicle_type'] = 1

mask = train_data['third_party_3_form'].isna()
train_data.loc[mask, 'third_party_3_form'] = 0
train_data.loc[~mask, 'third_party_3_form'] = 1

mask = train_data['third_party_3_year_birth'].isna()
train_data.loc[mask, 'third_party_3_year_birth'] = 0
train_data.loc[~mask, 'third_party_3_year_birth'] = 1

mask = train_data['third_party_3_country'].isna()
train_data.loc[mask, 'third_party_3_country'] = 0
train_data.loc[~mask, 'third_party_3_country'] = 1

mask = train_data['third_party_3_vehicle_id'].isna()
train_data.loc[mask, 'third_party_3_vehicle_id'] = 0
train_data.loc[~mask, 'third_party_3_vehicle_id'] = 1

mask = train_data['third_party_3_expert_id'].isna()
train_data.loc[mask, 'third_party_3_expert_id'] = 0
train_data.loc[~mask, 'third_party_3_expert_id'] = 1

In [9]:
# repair variables

train_data['repair_id'].fillna("MISSING", inplace=True)
repair_id_count = train_data['repair_id'].value_counts()
repair_id_count["MISSING"] = 0
train_data = train_data.merge(repair_id_count, how='left', 
                 left_on='repair_id', right_index=True)

train_data['repair_postal_code'].fillna(0, inplace=True)
repair_postal_code_count = train_data['repair_postal_code'].value_counts()
repair_postal_code_count[0] = 0
train_data = train_data.merge(repair_postal_code_count, how='left', 
                 left_on='repair_postal_code', right_index=True)

train_data['repair_form'].fillna("MISSING", inplace=True)

train_data['repair_year_birth'].fillna(1804, inplace=True)

train_data['repair_country'].fillna("MISSING", inplace=True)

In [10]:
# final variables

train_data['policy_date_start'].fillna(180501, inplace=True)
train_data['policy_date_start'] = pd.to_datetime(train_data['policy_date_start'].astype(int).astype(str),
                                                 format='%Y%m')

train_data['policy_date_next_expiry'].fillna(180501, inplace=True)
train_data['policy_date_next_expiry'] = pd.to_datetime(train_data['policy_date_next_expiry'].astype(int).astype(str),
                                                       format='%Y%m')

train_data['policy_date_last_renewed'].fillna(180501, inplace=True)
train_data['policy_date_last_renewed'] = pd.to_datetime(train_data['policy_date_last_renewed'].astype(int).astype(str),
                                                        format='%Y%m')

train_data['policy_premium_100'].fillna(200, inplace=True)

train_data['policy_coverage_1000'].fillna(300, inplace=True)

In [11]:
# drop cols for now

dropcols = ['claim_id', 'claim_postal_code_x', 'claim_vehicle_id_x',
       'policy_holder_id_x', 'policy_holder_postal_code_x',
       'policy_holder_expert_id_x', 'driver_id_x', 'driver_postal_code_x',
       'driver_expert_id_x', 'driver_vehicle_id_x', 'third_party_1_id_x',
       'third_party_1_postal_code_x', 'third_party_1_vehicle_id_x',
       'third_party_1_expert_id_x', 'repair_id_x', 'repair_postal_code_x',
       'claim_date_registered', 'claim_date_occured', 'claim_vehicle_date_inuse',
       'policy_date_start', 'policy_date_next_expiry', 'policy_date_last_renewed']

clean_data = train_data.drop(columns=dropcols)

In [12]:
# target and covariates

y = clean_data['fraud']
amount = clean_data['claim_amount']
X = clean_data.drop(columns=['fraud', 'claim_amount'])

In [13]:
# encode vars

encoded_covariates = pd.get_dummies(X)
encoded_target = pd.get_dummies(y)['Y']

In [14]:
# run model

rf = RandomForestClassifier()
rf.fit(encoded_covariates, encoded_target)

RandomForestClassifier()

In [15]:
rf.score(encoded_covariates, encoded_target)

1.0

In [16]:
# read test data

train_data = pd.read_csv('test.csv', delimiter=';')
train_data.shape

(29955, 76)

In [17]:
# claim variables

#train_data['claim_amount'] = train_data['claim_amount'].str.replace(',','.').astype('float64')

train_data['claim_date_registered'] = pd.to_datetime(train_data['claim_date_registered'], format='%Y%m%d')

train_data['claim_date_occured'] = pd.to_datetime(train_data['claim_date_occured'], format='%Y%m%d')

mask_night = (train_data['claim_time_occured'] >= 2200) | (train_data['claim_time_occured'] <= 700)
train_data.loc[~mask_night, 'claim_time_occured'] = 0
train_data.loc[mask_night, 'claim_time_occured'] = 1

postal_code_counts = train_data['claim_postal_code'].value_counts()
train_data = train_data.merge(postal_code_counts, how='left', left_on='claim_postal_code', right_index=True)

train_data['claim_alcohol'].fillna("MISSING", inplace=True)

train_data['claim_language'].fillna("MISSING", inplace=True)
mask = train_data['claim_language'] == 1.0
train_data.loc[mask, 'claim_language'] = "LANG A"
mask = train_data['claim_language'] == 2.0
train_data.loc[mask, 'claim_language'] = "LANG B"

train_data['claim_vehicle_id'].fillna("MISSING", inplace=True)
claim_vehicle_id_count = train_data['claim_vehicle_id'].value_counts()
claim_vehicle_id_count["MISSING"] = 0
train_data = train_data.merge(claim_vehicle_id_count, how='left', 
                              left_on='claim_vehicle_id', right_index=True)

train_data['claim_vehicle_brand'].fillna('MISSING', inplace=True)
claim_vehicle_brand_counts = train_data['claim_vehicle_brand'].value_counts()
claim_vehicle_brand_counts['MISSING'] = 0
train_data = train_data.merge(claim_vehicle_brand_counts, how='left', 
                              left_on='claim_vehicle_brand', right_index=True)

train_data['claim_vehicle_date_inuse'].fillna(190001, inplace=True)
train_data['claim_vehicle_date_inuse'] = train_data['claim_vehicle_date_inuse'].astype(int)
mask = (train_data['claim_vehicle_date_inuse'] > 220000)
train_data.loc[mask, 'claim_vehicle_date_inuse'] = 190001
train_data['claim_vehicle_date_inuse'] = pd.to_datetime(train_data['claim_vehicle_date_inuse'].astype(str), 
                                                        format='%Y%m')

train_data['claim_vehicle_cyl'].fillna(10000, inplace=True)

train_data['claim_vehicle_load'].fillna(500, inplace=True)

train_data['claim_vehicle_fuel_type'].fillna('MISSING', inplace=True)
mask = train_data['claim_vehicle_fuel_type'] == 1.0
train_data.loc[mask, 'claim_vehicle_fuel_type'] = "FUEL A"
mask = train_data['claim_vehicle_fuel_type'] == 2.0
train_data.loc[mask, 'claim_vehicle_fuel_type'] = "FUEL B"

train_data['claim_vehicle_power'].fillna(1000, inplace=True)

In [18]:
# policy variables

policy_holder_id_count = train_data['policy_holder_id'].value_counts()
train_data = train_data.merge(policy_holder_id_count, how='left', 
                              left_on='policy_holder_id', right_index=True)

train_data['policy_holder_postal_code'].fillna(0, inplace=True)
policy_holder_postal_code_counts = train_data['policy_holder_postal_code'].value_counts()
policy_holder_postal_code_counts.loc[0] = 0
train_data = train_data.merge(policy_holder_postal_code_counts, how='left', 
                              left_on='policy_holder_postal_code', right_index=True)

train_data['policy_holder_year_birth'].fillna(1800, inplace=True)

train_data['policy_holder_expert_id'].fillna("MISSING", inplace=True)
policy_holder_expert_id_count = train_data['policy_holder_expert_id'].value_counts()
policy_holder_expert_id_count['MISSING'] = 0
train_data = train_data.merge(policy_holder_expert_id_count, how='left', 
                              left_on='policy_holder_expert_id', right_index=True)

In [19]:
# driver variables

driver_id_count = train_data['driver_id'].value_counts()
train_data = train_data.merge(driver_id_count, how='left', 
                              left_on='driver_id', right_index=True)

train_data['driver_postal_code'].fillna(0, inplace=True)
driver_postal_code_count = train_data['driver_postal_code'].value_counts()
driver_postal_code_count.loc[0] = 0
train_data = train_data.merge(driver_postal_code_count, how='left', 
                              left_on='driver_postal_code', right_index=True)

train_data['driver_year_birth'].fillna(1801, inplace=True)

train_data['driver_expert_id'].fillna("MISSING", inplace=True)
driver_expert_id_count = train_data['driver_expert_id'].value_counts()
driver_expert_id_count['MISSING'] = 0
train_data = train_data.merge(driver_expert_id_count, how='left', 
                              left_on='driver_expert_id', right_index=True)

train_data['driver_vehicle_id'].fillna("MISSING", inplace=True)
driver_vehicle_id_count = train_data['driver_vehicle_id'].value_counts()
driver_vehicle_id_count["MISSING"] = 0
train_data = train_data.merge(driver_vehicle_id_count, how='left', 
                              left_on='driver_vehicle_id', right_index=True)

In [20]:
# third party 1st variables

train_data['third_party_1_id'].fillna("MISSING", inplace=True)
third_party_1_id_count = train_data['third_party_1_id'].value_counts()
third_party_1_id_count["MISSING"] = 0
train_data = train_data.merge(third_party_1_id_count, how='left', 
                              left_on='third_party_1_id', right_index=True)

train_data['third_party_1_postal_code'].fillna(0, inplace=True)
third_party_1_postal_code_count = train_data['third_party_1_postal_code'].value_counts()
third_party_1_postal_code_count[0] = 0
train_data = train_data.merge(third_party_1_postal_code_count, how='left', 
                              left_on='third_party_1_postal_code', right_index=True)

train_data['third_party_1_injured'].fillna("MISSING", inplace=True)

train_data['third_party_1_vehicle_type'].fillna("MISSING", inplace=True)

train_data['third_party_1_form'].fillna("MISSING", inplace=True)

train_data['third_party_1_year_birth'].fillna(1802, inplace=True)

train_data['third_party_1_country'].fillna("MISSING", inplace=True)

train_data['third_party_1_vehicle_id'].fillna("MISSING", inplace=True)
third_party_1_vehicle_id_count = train_data['third_party_1_vehicle_id'].value_counts()
third_party_1_vehicle_id_count["MISSING"] = 0
train_data = train_data.merge(third_party_1_vehicle_id_count, how='left', 
                              left_on='third_party_1_vehicle_id', right_index=True)

train_data['third_party_1_expert_id'].fillna("MISSING", inplace=True)
third_party_1_expert_id_count = train_data['third_party_1_expert_id'].value_counts()
third_party_1_expert_id_count["MISSING"] = 0
train_data = train_data.merge(third_party_1_expert_id_count, how='left', 
                 left_on='third_party_1_expert_id', right_index=True)

In [21]:
# third party rest of variables

mask = train_data['third_party_2_id'].isna()
train_data.loc[mask, 'third_party_2_id'] = 0
train_data.loc[~mask, 'third_party_2_id'] = 1

mask = train_data['third_party_2_postal_code'].isna()
train_data.loc[mask, 'third_party_2_postal_code'] = 0
train_data.loc[~mask, 'third_party_2_postal_code'] = 1

mask = train_data['third_party_2_injured'].isna()
train_data.loc[mask, 'third_party_2_injured'] = 0
train_data.loc[~mask, 'third_party_2_injured'] = 1

mask = train_data['third_party_2_vehicle_type'].isna()
train_data.loc[mask, 'third_party_2_vehicle_type'] = 0
train_data.loc[~mask, 'third_party_2_vehicle_type'] = 1

mask = train_data['third_party_2_form'].isna()
train_data.loc[mask, 'third_party_2_form'] = 0
train_data.loc[~mask, 'third_party_2_form'] = 1

mask = train_data['third_party_2_year_birth'].isna()
train_data.loc[mask, 'third_party_2_year_birth'] = 0
train_data.loc[~mask, 'third_party_2_year_birth'] = 1

mask = train_data['third_party_2_country'].isna()
train_data.loc[mask, 'third_party_2_country'] = 0
train_data.loc[~mask, 'third_party_2_country'] = 1

mask = train_data['third_party_2_vehicle_id'].isna()
train_data.loc[mask, 'third_party_2_vehicle_id'] = 0
train_data.loc[~mask, 'third_party_2_vehicle_id'] = 1

mask = train_data['third_party_2_expert_id'].isna()
train_data.loc[mask, 'third_party_2_expert_id'] = 0
train_data.loc[~mask, 'third_party_2_expert_id'] = 1

mask = train_data['third_party_3_id'].isna()
train_data.loc[mask, 'third_party_3_id'] = 0
train_data.loc[~mask, 'third_party_3_id'] = 1

mask = train_data['third_party_3_postal_code'].isna()
train_data.loc[mask, 'third_party_3_postal_code'] = 0
train_data.loc[~mask, 'third_party_3_postal_code'] = 1

mask = train_data['third_party_3_injured'].isna()
train_data.loc[mask, 'third_party_3_injured'] = 0
train_data.loc[~mask, 'third_party_3_injured'] = 1

mask = train_data['third_party_3_vehicle_type'].isna()
train_data.loc[mask, 'third_party_3_vehicle_type'] = 0
train_data.loc[~mask, 'third_party_3_vehicle_type'] = 1

mask = train_data['third_party_3_form'].isna()
train_data.loc[mask, 'third_party_3_form'] = 0
train_data.loc[~mask, 'third_party_3_form'] = 1

mask = train_data['third_party_3_year_birth'].isna()
train_data.loc[mask, 'third_party_3_year_birth'] = 0
train_data.loc[~mask, 'third_party_3_year_birth'] = 1

mask = train_data['third_party_3_country'].isna()
train_data.loc[mask, 'third_party_3_country'] = 0
train_data.loc[~mask, 'third_party_3_country'] = 1

mask = train_data['third_party_3_vehicle_id'].isna()
train_data.loc[mask, 'third_party_3_vehicle_id'] = 0
train_data.loc[~mask, 'third_party_3_vehicle_id'] = 1

mask = train_data['third_party_3_expert_id'].isna()
train_data.loc[mask, 'third_party_3_expert_id'] = 0
train_data.loc[~mask, 'third_party_3_expert_id'] = 1

In [22]:
# repair variables

train_data['repair_id'].fillna("MISSING", inplace=True)
repair_id_count = train_data['repair_id'].value_counts()
repair_id_count["MISSING"] = 0
train_data = train_data.merge(repair_id_count, how='left', 
                 left_on='repair_id', right_index=True)

train_data['repair_postal_code'].fillna(0, inplace=True)
repair_postal_code_count = train_data['repair_postal_code'].value_counts()
repair_postal_code_count[0] = 0
train_data = train_data.merge(repair_postal_code_count, how='left', 
                 left_on='repair_postal_code', right_index=True)

train_data['repair_form'].fillna("MISSING", inplace=True)

train_data['repair_year_birth'].fillna(1804, inplace=True)

train_data['repair_country'].fillna("MISSING", inplace=True)

In [23]:
# final variables

train_data['policy_date_start'].fillna(180501, inplace=True)
train_data['policy_date_start'] = pd.to_datetime(train_data['policy_date_start'].astype(int).astype(str),
                                                 format='%Y%m')

train_data['policy_date_next_expiry'].fillna(180501, inplace=True)
train_data['policy_date_next_expiry'] = pd.to_datetime(train_data['policy_date_next_expiry'].astype(int).astype(str),
                                                       format='%Y%m')

train_data['policy_date_last_renewed'].fillna(180501, inplace=True)
train_data['policy_date_last_renewed'] = pd.to_datetime(train_data['policy_date_last_renewed'].astype(int).astype(str),
                                                        format='%Y%m')

train_data['policy_premium_100'].fillna(200, inplace=True)

train_data['policy_coverage_1000'].fillna(300, inplace=True)

In [24]:
# drop cols for now

dropcols = ['claim_id', 'claim_postal_code_x', 'claim_vehicle_id_x',
       'policy_holder_id_x', 'policy_holder_postal_code_x',
       'policy_holder_expert_id_x', 'driver_id_x', 'driver_postal_code_x',
       'driver_expert_id_x', 'driver_vehicle_id_x', 'third_party_1_id_x',
       'third_party_1_postal_code_x', 'third_party_1_vehicle_id_x',
       'third_party_1_expert_id_x', 'repair_id_x', 'repair_postal_code_x',
       'claim_date_registered', 'claim_date_occured', 'claim_vehicle_date_inuse',
       'policy_date_start', 'policy_date_next_expiry', 'policy_date_last_renewed']

clean_data = train_data.drop(columns=dropcols)

In [25]:
# target and covariates

#y = clean_data['fraud']
#amount = clean_data['claim_amount']
X = clean_data

In [26]:
X.head()

,claim_time_occured,claim_cause,claim_liable,claim_num_injured,claim_num_third_parties,claim_num_vehicles,claim_police,claim_alcohol,claim_language,claim_vehicle_brand_x,...,driver_id_y,driver_postal_code_y,driver_expert_id_y,driver_vehicle_id_y,third_party_1_id_y,third_party_1_postal_code_y,third_party_1_vehicle_id_y,third_party_1_expert_id_y,repair_id_y,repair_postal_code_y
0,0.0,traffic accident,Y,0,1,2,N,MISSING,LANG B,AUDI,...,1,45,0,1,1,31,1,0,31,35
1,0.0,traffic accident,Y,0,1,3,N,MISSING,LANG B,FORD,...,2,400,0,1,2,24,1,0,0,0
2,0.0,traffic accident,Y,0,1,2,N,MISSING,LANG B,HYUNDAI,...,5,70,0,1,1,28,1,0,0,0
3,0.0,traffic accident,Y,0,1,3,N,MISSING,LANG A,RENAULT,...,1,8,0,1,4,26,1,0,0,0
4,0.0,traffic accident,Y,0,1,2,N,MISSING,LANG A,VOLKSWAGEN,...,1,17,0,1,1,19,1,0,0,0


In [27]:
# encode vars

encoded_covariates = pd.get_dummies(X)
#encoded_target = pd.get_dummies(y)['Y']

In [28]:
encoded_covariates.columns

Index(['claim_time_occured', 'claim_num_injured', 'claim_num_third_parties',
       'claim_num_vehicles', 'claim_vehicle_cyl', 'claim_vehicle_load',
       'claim_vehicle_power', 'policy_holder_year_birth', 'driver_year_birth',
       'third_party_1_year_birth',
       ...
       'policy_coverage_type_#111110010', 'policy_coverage_type_#111110011',
       'policy_coverage_type_#111110100', 'policy_coverage_type_#111110110',
       'policy_coverage_type_#111111000', 'policy_coverage_type_#111111001',
       'policy_coverage_type_#111111010', 'policy_coverage_type_#111111011',
       'policy_coverage_type_#111111100', 'policy_coverage_type_#111111110'],
      dtype='object', length=233)

In [38]:
predictions = rf.predict_proba(encoded_covariates)

In [39]:
predictions = pd.DataFrame(predictions, index=train_data['claim_id'])
predictions.head()

,0,1
claim_id,,
65469,0.99,0.01
65470,0.99,0.01
65471,0.97,0.03
65472,0.96,0.04
65473,0.99,0.01


In [41]:
predictions[1].to_csv('test_proba.csv', index=True)